In [3]:
include("experiments/experiments.jl")

analyze_domain_results

In [4]:
print(EXPERIMENTS_PATH)

./experiments

In [5]:
run_domain_experiments(EXPERIMENTS_PATH, "taxi", "optimal", :sips)

Running experiments for problem 0...
Inferring goals for trajectory 0, goal 0...
True goal: and(passenger-at(green))
Running reimplementation of particle filter for SIPS
Timestep 1
Timestep 2
Timestep 3
Timestep 4
Timestep 5
Timestep 6
Timestep 7
Timestep 8
Timestep 9
Timestep 10
Timestep 11
Timestep 12
Timestep 13
Writing results to taxi_problem_0_goal0.csv...
Inferring goals for trajectory 1, goal 1...
True goal: and(passenger-at(yellow))
Running reimplementation of particle filter for SIPS
Timestep 1
Timestep 2
Timestep 3
Timestep 4
Timestep 5
Timestep 6
Timestep 7
Timestep 8
Timestep 9
Writing results to taxi_problem_0_goal1.csv...
Inferring goals for trajectory 2, goal 2...
True goal: and(passenger-at(blue))
Running reimplementation of particle filter for SIPS
Timestep 1
Timestep 2
Timestep 3
Timestep 4
Timestep 5
Timestep 6
Timestep 7
Timestep 8
Timestep 9
Timestep 10
Timestep 11
Timestep 12
Writing results to taxi_problem_0_goal2.csv...
Writing summary results to taxi_problem_0_

LoadError: InterruptException: